# Word Embeddings Première étape : la data préparation

In [1]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 8.8 MB/s eta 0:00:00


In [8]:
import re
import nltk
import emoji
import numpy as np
from nltk.tokenize import word_tokenize

from scipy import linalg
from collections import defaultdict

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [9]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Data préparation

Dans la phase de préparation des données, en commençant par un corpus de texte, vous:

- Nettoyer et marquer le corpus.

- Extraire les paires de mots de contexte et de mot central qui constitueront l'ensemble des données d'entraînement pour le modèle CBOW. Les mots contextuels sont les caractéristiques qui seront introduites dans le modèle, et les mots centraux sont les valeurs cibles que le modèle apprendra à prédire.

- Créez des représentations vectorielles simples des mots de contexte (caractéristiques) et des mots centraux (cibles) qui peuvent être utilisés par le réseau neuronal du modèle CBOW.

## Cleaning et tokenization

Pour démontrer le processus de nettoyage et de tokenisation, considérons un corpus qui contient des emojis et divers signes de ponctuation.

In [20]:
# Define a corpus
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!'

Premièrement, remplacez tous les signes de ponctuation interruptifs - tels que les virgules et les points d'exclamation - par des points.

In [21]:
# Print original corpus
print(f'Corpus:  {corpus}')

# Do the substitution
### A modifier - debut
data = re.sub(r'[!?]+', '.', corpus)
### A modifier - fin

# Print cleaned corpus
print(f'After cleaning punctuation:  {data}')

Corpus:  Who ❤️ "word embeddings" in 2020? I do!!!
After cleaning punctuation:  Who ❤️ "word embeddings" in 2020. I do.


Ensuite, utilisez le moteur de tokenisation de NLTK pour diviser le corpus en tokens individuels.

In [22]:
# Print cleaned corpus
print(f'Initial string:  {data}')

# Tokenize the cleaned corpus
### A modifier - debut
data = word_tokenize(data)
### A modifier - fin

# Print the tokenized version of the corpus
print(f'After tokenization:  {data}')

Initial string:  Who ❤️ "word embeddings" in 2020. I do.
After tokenization:  ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']


Enfin, comme vous l'avez vu dans le cours, débarrassez-vous des chiffres et de la ponctuation autres que les points, et convertissez tous les tokens restants en minuscules.


In [23]:
# Print the tokenized version of the corpus
print(f'Initial list of tokens:  {data}')

# Filter tokenized corpus using list comprehension
### A modifier - debut
# Éliminer les chiffres et la ponctuation autre que les points, et convertir en minuscules
data = [re.sub(r'(?<!\S)[^a-z\.\s❤️]+(?!\S)', '', token.lower()) for token in data]

# Filtrer les tokens vides résultant du nettoyage
data = [token for token in data if token]
### A modifier - fin

# Print the tokenized and filtered version of the corpus
print(f'After cleaning:  {data}')

Initial list of tokens:  ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']
After cleaning:  ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do', '.']


Notez que l'émoji du coeur est considéré comme un token comme n'importe quel mot normal.

Maintenant, rationalisons le processus de nettoyage et de tokenisation en intégrant les étapes précédentes dans une fonction.

In [24]:
# Define the 'tokenize' function that will include the steps previously seen
def tokenize(corpus):

    # Tokeniser le corpus en utilisant NLTK
    tokens = word_tokenize(corpus)

    # Nettoyer les tokens en éliminant les chiffres et la ponctuation autre que les points et les emojis, et convertir en minuscules
    data = [
        re.sub(r'(?<!\S)[^a-z\.\s❤️]+(?!\S)', '', token.lower())
        for token in tokens
        if not re.match(r'^\d+$', token)
    ]

    # Filtrer les tokens vides résultant du nettoyage
    data = [token for token in data if token]
    return data

Appliquez cette fonction au corpus sur lequel vous allez travailler dans le reste de ce carnet : "I am happy because I am learning"

In [25]:
# Define new corpus
corpus = 'I am happy because I am learning'

# Print new corpus
print(f'Corpus:  {corpus}')

# Save tokenized version of corpus into 'words' variable
words = tokenize(corpus)

# Print the tokenized version of the corpus
print(f'Words (tokens):  {words}')

Corpus:  I am happy because I am learning
Words (tokens):  ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


**Maintenant, essayez-le vous-même avec votre propre phrase.**

In [26]:
# Run this with any sentence
tokenize("Now it's your turn: try with your own sentence!")


['now', 'it', "'s", 'your', 'turn', 'try', 'with', 'your', 'own', 'sentence']

## Sliding window of words

Maintenant que vous avez transformé le corpus en une liste de token propres, vous pouvez faire glisser une fenêtre de mots sur cette liste. Pour chaque fenêtre, vous pouvez extraire un mot central et les mots de contexte.

La fonction `get_windows` dans la cellule suivante a été introduite dans le cours.

Le premier argument de cette fonction est une liste de mots (ou de tokens). Le deuxième argument, "C", est le contexte demi-taille. Rappelons que pour un mot central donné, les mots de contexte sont constitués de mots "C" à gauche et de mots "C" à droite du mot central.


In [28]:
# Define the 'get_windows' function
def get_windows(words, C):
    i = C
    while i < len(words) - C:

        ### A modifier - debut
        center_word = words[i]
        context_words = words[i - C:i] + words[i + 1:i + C + 1]
        ### A modifier - fin
        yield context_words, center_word
        i += 1

Voici comment vous pouvez utiliser cette fonction pour extraire les mots de contexte et les mots centraux d'une liste de tokens. Ces mots de contexte et mots centraux constitueront le jeu de formation que vous utiliserez pour former le modèle CBOW.

In [29]:
# Print 'context_words' and 'center_word' for the new corpus with a 'context half-size' of 2
for x, y in get_windows(['i', 'am', 'happy', 'because', 'i', 'am', 'learning'], 2):
    print(f'{x}\t{y}')

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i


Le premier exemple de l'ensemble de formation est constitué de

- les mots de contexte "I", "am", "because", "I",

- et le mot central à prédire : "happy".

**Maintenant, essayez-le vous-même. Dans la cellule suivante, vous pouvez changer la phrase et l'hyperparamètre C.

In [39]:
# Print 'context_words' and 'center_word' for any sentence with a 'context half-size' of 1
for x, y in get_windows(tokenize("This year at Christmas i'd got : a car, and some money!"), 2):
    print(f'{x}\t{y}')

['this', 'year', 'christmas', 'i']	at
['year', 'at', 'i', "'d"]	christmas
['at', 'christmas', "'d", 'got']	i
['christmas', 'i', 'got', 'a']	'd
['i', "'d", 'a', 'car']	got
["'d", 'got', 'car', 'and']	a
['got', 'a', 'and', 'some']	car
['a', 'car', 'some', 'money']	and


## Transformer les mots en vecteurs pour l'entraînement

Pour terminer la préparation de l'ensemble du jeu d'entraînement, il faut transformer les mots de contexte et les mots centraux en vecteurs.

### Mise en correspondance des mots avec les indices et des indices avec les mots

Les mots centraux seront représentés sous forme de vecteurs one-hot, et les vecteurs qui représentent les mots de contexte sont également basés sur des vecteurs one-hot.

Pour créer des vecteurs de mots uniques, vous pouvez commencer par faire correspondre chaque mot unique à un entier (ou index) unique. Je vous ai fourni une fonction d'aide, `get_dict`, qui crée un dictionnaire Python qui fait correspondre les mots à des entiers et vice-versa.



In [33]:
def get_dict(data):
   """    Input:
           K: the number of negative samples
           data: the data you want to pull from
           indices: a list of word indices
         Output:
           word_dict: a dictionary with the weighted probabilities of each word
           word2Ind: returns dictionary mapping the word to its index
           Ind2Word: returns dictionary mapping the index to its word
   """

   words = sorted(list(set(data)))
   n = len(words)
   idx = 0

   # return these correctly
   word2Ind = {}
   Ind2word = {}
   for k in words:
     word2Ind[k] = idx
     Ind2word[idx] = k
     idx += 1
   return word2Ind, Ind2word

In [34]:
# Get 'word2Ind' and 'Ind2word' dictionaries for the tokenized corpus
word2Ind, Ind2word = get_dict(words)

Voici le dictionnaire qui fait correspondre les mots à des indices numériques.

In [35]:
# Print 'word2Ind' dictionary
word2Ind

{'am': 0, 'because': 1, 'happy': 2, 'i': 3, 'learning': 4}

Vous pouvez utiliser ce dictionnaire pour obtenir l'index d'un mot.

In [40]:
# Print value for the key 'i' within word2Ind dictionary
print("Index of the word 'i':  ",word2Ind['i'])

Index of the word 'i':   3


Et inversement, voici le dictionnaire qui fait correspondre les index aux mots.

In [41]:
# Print 'Ind2word' dictionary
Ind2word

{0: 'am', 1: 'because', 2: 'happy', 3: 'i', 4: 'learning'}

In [42]:
# Print value for the key '2' within Ind2word dictionary
print("Word which has index 2:  ",Ind2word[2] )

Word which has index 2:   happy


Enfin, obtenez la longueur de l'un ou l'autre de ces dictionnaires pour connaître la taille du vocabulaire de votre corpus, c'est-à-dire le nombre de mots différents composant le corpus.

In [45]:
# Save length of word2Ind dictionary into the 'V' variable
V = len(word2Ind)

# Print length of word2Ind dictionary
print("Size of vocabulary: ", V)

Size of vocabulary:  5


### Obtenir des one-hot vecteurs

Rappelez-vous que vous pouvez facilement convertir un entier, $n$, en un vecteur d'un seul coup.

Considérez le mot "happy". Tout d'abord, récupérez son index numérique.

In [46]:
# Save index of word 'happy' into the 'n' variable
n = word2Ind['happy']

# Print index of word 'happy'
n

2

Créez maintenant un vecteur de la taille du vocabulaire, et remplissez-le avec des zéros.

In [47]:
# Create vector with the same length as the vocabulary, filled with zeros
### A modifier - debut
center_word_vector = np.zeros(V)
### A modifier - fin

# Print vector
center_word_vector

array([0., 0., 0., 0., 0.])

Vous pouvez confirmer que le vecteur a la bonne taille.

In [48]:
# Assert that the length of the vector is the same as the size of the vocabulary
len(center_word_vector) == V

True

Ensuite, remplacez le 0 de l'élément $n$-th par un 1.

In [49]:
# Replace element number 'n' with a 1
center_word_vector[n] = 1

Et vous avez votre one-hot vecteur pour votre mot.

In [50]:
# Print vector
center_word_vector

array([0., 0., 1., 0., 0.])

**Vous pouvez maintenant regrouper toutes ces étapes dans une fonction pratique, qui prend comme paramètres : un mot à encoder, un dictionnaire qui associe les mots à des index, et la taille du vocabulaire.**

In [51]:
# Define the 'word_to_one_hot_vector' function that will include the steps previously seen
def word_to_one_hot_vector(word, word2Ind, V):

    ### A modifier - debut
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2Ind[word]] = 1
    ### A modifier - fin
    return one_hot_vector

Vérifiez qu'il fonctionne comme prévu.

In [52]:
# Print output of 'word_to_one_hot_vector' function for word 'happy'
word_to_one_hot_vector('happy', word2Ind, V)

array([0., 0., 1., 0., 0.])

**Quel est le mot vecteur pour "learning"?**

In [53]:
# Print output of 'word_to_one_hot_vector' function for word 'learning'
word_to_one_hot_vector('learning', word2Ind, V)

array([0., 0., 0., 0., 1.])

Réponse attendue :

    array([0., 0., 0., 0., 1.])

### Obtenir des vecteurs de mots contextuels

Pour créer les vecteurs qui représentent les mots de contexte, vous calculerez la moyenne des one-hot vecteurs représentant les mots individuels.

Commençons par une liste de mots contextuels.

In [54]:
# Define list containing context words
context_words = ['i', 'am', 'because', 'i']

En utilisant la fonction `word_to_one_hot_vector` que vous avez créée dans la section précédente, vous pouvez créer une liste de vecteurs one-hot représentant chacun des mots du contexte.

In [57]:
# Create one-hot vectors for each context word using list comprehension

### A modifier - debut
context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
### A modifier - fin

# Print one-hot vectors for each context word
context_words_vectors

[array([0., 0., 0., 1., 0.]),
 array([1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0.])]

Et vous pouvez maintenant simplement obtenir la moyenne de ces vecteurs en utilisant la fonction `mean` de numpy, pour obtenir la représentation vectorielle des mots de contexte.

In [58]:
# Compute mean of the vectors using numpy
np.mean(context_words_vectors, axis=0)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

Notez le paramètre `axis=0` de la fonction `mean` est utilisé pour calculer la moyenne des lignes (si vous aviez voulu la moyenne des colonnes, vous auriez utilisé `axis=1`).

**Créez maintenant la fonction `context_words_to_vector` qui prend en compte une liste de mots de contexte, un dictionnaire de mots à indexer, et une taille de vocabulaire, et produit la représentation vectorielle des mots de contexte.**

In [59]:
# Define the 'context_words_to_vector' function that will include the steps previously seen
def context_words_to_vector(context_words, word2Ind, V):

    ### A modifier - debut
    context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
    context_words_vectors = np.mean(context_words_vectors, axis=0)
    ### A modifier - fin
    return context_words_vectors

Et vérifiez que vous obtenez le même résultat que l'approche manuelle ci-dessus.

In [60]:
# Print output of 'context_words_to_vector' function for context words: 'i', 'am', 'because', 'i'
context_words_to_vector(['i', 'am', 'because', 'i'], word2Ind, V)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

**Quelle est la représentation vectorielle des mots du contexte "am happy i am" ?**

In [61]:
# Print output of 'context_words_to_vector' function for context words: 'am', 'happy', 'i', 'am'
context_words_to_vector(['am', 'happy', 'i', 'am'], word2Ind, V)

array([0.5 , 0.  , 0.25, 0.25, 0.  ])

Réponse attendue:

    array([0.5 , 0.  , 0.25, 0.25, 0.  ])

## Construire l'ensemble du training set

Vous pouvez maintenant combiner les fonctions que vous avez créées dans les sections précédentes, afin de construire le jeu d'entraînement pour le modèle CBOW, à partir du corpus suivant.

In [62]:
# Print corpus
words

['i', 'am', 'happy', 'because', 'i', 'am', 'learning']

Pour ce faire, vous devez utiliser la fonction de fenêtre coulissante (`get_windows`) pour extraire les mots de contexte et les mots de centre, et vous devez ensuite convertir ces ensembles de mots en une représentation vectorielle de base en utilisant `word_to_one_hot_vector` et `context_words_to_vector`.

In [65]:
# Print vectors associated to center and context words for corpus
for context_words, center_word in get_windows(words, 2):  # reminder: 2 is the context half-size
    print(f'Context words:  {context_words} -> {context_words_to_vector(context_words, word2Ind, V)}')
    print(f'Center word:  {center_word} -> {word_to_one_hot_vector(center_word, word2Ind, V)}')
    print()

Context words:  ['i', 'am', 'because', 'i'] -> [0.25 0.25 0.   0.5  0.  ]
Center word:  happy -> [0. 0. 1. 0. 0.]

Context words:  ['am', 'happy', 'i', 'am'] -> [0.5  0.   0.25 0.25 0.  ]
Center word:  because -> [0. 1. 0. 0. 0.]

Context words:  ['happy', 'because', 'am', 'learning'] -> [0.25 0.25 0.25 0.   0.25]
Center word:  i -> [0. 0. 0. 1. 0.]



Dans ce notebook, vous effectuerez une seule itération du jeu d'entraînement en utilisant un seul exemple, mais dans le cadre d'un projet, vous formerez le modèle CBOW en utilisant plusieurs itérations et différent batchs d'exemples.

Voici comment vous utiliserez une fonction de générateur Python (rappelez-vous le mot-clé "[yield](https://fr.wikipedia.org/wiki/Yield_(instruction))") pour faciliter l'itération sur un ensemble d'exemples.

In [68]:
# Define the generator function 'get_training_example'
def get_training_example(words, C, word2Ind, V):
    ### A modifier - debut
    for context_words, center_word in get_windows(words, C):
        yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)
    ### A modifier - fin

La sortie de cette fonction peut être répétée pour obtenir des vecteurs de mots de contexte et des vecteurs de mots centraux successifs, comme démontré dans la cellule suivante.

In [69]:
# Print vectors associated to center and context words for corpus using the generator function
for context_words_vector, center_word_vector in get_training_example(words, 2, word2Ind, V):
    print(f'Context words vector:  {context_words_vector}')
    print(f'Center word vector:  {center_word_vector}')
    print()


Context words vector:  [0.25 0.25 0.   0.5  0.  ]
Center word vector:  [0. 0. 1. 0. 0.]

Context words vector:  [0.5  0.   0.25 0.25 0.  ]
Center word vector:  [0. 1. 0. 0. 0.]

Context words vector:  [0.25 0.25 0.25 0.   0.25]
Center word vector:  [0. 0. 0. 1. 0.]



Votre jeu d'entraînement est prêt, vous pouvez maintenant passer au modèle CBOW.
